# Neural Networks with keras and tensorflow

N.B. You will need to pip install keras and tensorflow!

In [1]:
from sklearn.datasets import load_breast_cancer
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [3]:
#!pip install tensorflow==1.14.0

import tensorflow as tf

For this lesson we'll use sklearn's built-in breast cancer dataset. The next cell loads the data and prints the data description.

In [4]:
data = load_breast_cancer()
print(data.DESCR)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target)

ss = StandardScaler()
ss.fit(X_train)

In [7]:
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

In [10]:
X_train_scaled.shape

## Constructing a Neural Network in Keras

In [12]:
# Importing model and layer types

from keras.models import Sequential
from keras.layers import Dense

# Importing our optimizer

from keras.optimizers import Adam

In [14]:
model = Sequential()

inputs = X_train.shape[1]
hiddens = inputs

model.add(Dense(hiddens, input_dim=inputs, activation='tanh'))
model.add(Dense(1))

In [16]:
adam = Adam()
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['acc'])

In [18]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),
         epochs=20)

In [19]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=50, verbose=0, batch_size=200)

What is the difference between "validation data" and "test data"? Strictly speaking (though not all DS practitioners speak strictly), validation data is used to choose models' hyperparameters while test data is used to evaluate a fully-developed model. For more, see [here](https://machinelearningmastery.com/difference-test-validation-datasets/).

In [21]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']
plt.plot(train_loss, label='Training Loss')
plt.plot(test_loss, label='Testing Loss')
plt.legend();

In [23]:
np.hstack([model.predict(X_test), y_test.reshape(143, 1)])[:20]

In [25]:
model.evaluate(X_test, y_test)

## Tensorflow

### Tensorflow as a dynamic programming tool

In [26]:
slot_1 = tf.compat.v1.placeholder(tf.int64, shape=(3, 2))

slot_2 = tf.compat.v1.placeholder(tf.int64, shape=(2, 3))

ans = tf.matmul(slot_1, slot_2)

In [27]:
with tf.Session() as sess:
    result = sess.run(ans, feed_dict={slot_1:[[3, 3],[2, 2],[1, 1]], slot_2:[[1, 2, 3], [1, 1, 1]]})

In [49]:
print(result)

### Neural Network

In [9]:
tf.reset_default_graph()

X = tf.compat.v1.placeholder(dtype=tf.float32, shape=(None, 30))
y = tf.compat.v1.placeholder(dtype=tf.float32, shape=(None, 1))

hid = tf.layers.dense(X, 30, activation=tf.nn.tanh)
y_hat = tf.layers.dense(hid, 1, activation=tf.nn.tanh)

loss = tf.losses.log_loss(y, y_hat)
optimizer = tf.train.AdamOptimizer(0.01)
training_run = optimizer.minimize(loss)

saver = tf.train.Saver()

In [13]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for _ in range(100):
        sess.run(training_run, feed_dict={X: X_train_scaled, y: y_train.reshape(-1, 1)})
        
    pred = sess.run(y_hat, feed_dict={X: X_test})

classes = (pred > 0.5).astype(int)

print(metrics.mean_squared_error(y_test, classes))
print(metrics.accuracy_score(y_test, classes))